<a href="https://colab.research.google.com/github/keerthana2610/Intro-to-ML-Project/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #mounting the given data file
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


Importing Libraries

In [ ]:
import os
import ast
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.patches as mpatches
import seaborn as sn
import math
from tqdm.std import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
class Config:
    data_dir = '../data/'
    working_dir = '../src/'
    models_dir = '../models/'
    submissions_dir = '../submissions'

Reading datasets

In [ ]:
#creating dataframe for given datasets
train = pd.read_csv("Train.csv")
test=pd.read_csv("Test.csv")
mdata=pd.read_csv("metadata.csv")
ss=pd.read_csv("SampleSubmission.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
mdata.head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_K00S4N4,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30
1,ID_6L67PAA,2015-12-09 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740.0,2016-12-07 00:00:00,2015-12-09 13:14:03,2020-05-24 15:32:18
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52


In [ ]:
mdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37343 entries, 0 to 37342
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   37343 non-null  object 
 1   RegistrationDate     37343 non-null  object 
 2   Deposit              37343 non-null  int64  
 3   UpsellDate           973 non-null    object 
 4   AccessoryRate        37343 non-null  float64
 5   PaymentMethod        37343 non-null  object 
 6   rateTypeEntity       37343 non-null  object 
 7   RatePerUnit          37343 non-null  int64  
 8   DaysOnDeposit        37343 non-null  int64  
 9   MainApplicantGender  37343 non-null  object 
 10  Age                  30404 non-null  float64
 11  Region               35409 non-null  object 
 12  Town                 37343 non-null  object 
 13  Occupation           37343 non-null  object 
 14  SupplierName         37343 non-null  object 
 15  Term                 37343 non-null 

In [ ]:
test.head(5)

,ID,TransactionDates,PaymentsHistory
0,ID_6L67PAA,"['12-2015', '01-2016', '02-2016', '03-2016', '05-2016', '07-2016']","[4000.0, 1050.0, 1050.0, 1050.0, 1050.0, 400.0]"
1,ID_VJ80SX2,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016']","[3000.0, 850.0, 750.0, 1500.0, 650.0, 1250.0, 1000.0]"
2,ID_7OU9HLK,"['12-2015', '01-2016', '03-2016', '05-2016', '11-2016', '12-2016', '01-2017', '02-2017']","[2400.0, 300.0, 500.0, 450.0, 675.0, 700.0, 875.0, 175.0]"
3,ID_WVWTPGK,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016', '11-2016']","[4700.0, 1200.0, 950.0, 1200.0, 900.0, 1110.0, 900.0, 1010.0, 1015.0, 925.0, 800.0, 485.0]"
4,ID_04DSDQS,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016']","[4800.0, 750.0, 995.0, 995.0, 1300.0, 750.0, 1000.0, 500.0, 500.0]"


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9336 entries, 0 to 9335
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                9336 non-null   object
 1   TransactionDates  9336 non-null   object
 2   PaymentsHistory   9336 non-null   object
dtypes: object(3)
memory usage: 218.9+ KB


In [ ]:
train.head(5)

,ID,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_MR53LEX,"['04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '09-2018', '10-2018', '11-2018', '12-2018', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020']","[3600.0, 750.0, 350.0, 65.0, 95.0, 135.0, 85.0, 55.0, 345.0, 155.0, 55.0, 630.0, 585.0, 185.0, 440.0, 483.0, 660.0, 515.0, 292.0, 505.0, 210.0, 120.0, 260.0, 610.0, 230.0, 390.0, 660.0, 1200.0, 1320.0, 280.0, 770.0]",880.0,930.0,495.0,715.0,220.0,385.0
1,ID_3D7NQUH,"['04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '11-2018', '12-2018', '01-2019', '02-2019', '03-2019', '04-2019', '05-2019', '06-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019', '12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020']","[2940.0, 970.0, 380.0, 880.0, 385.0, 440.0, 1100.0, 605.0, 550.0, 605.0, 715.0, 935.0, 880.0, 660.0, 770.0, 440.0, 825.0, 770.0, 605.0, 639.0, 655.0, 605.0, 495.0, 442.0, 495.0, 605.0, 660.0, 660.0, 655.0, 770.0]",660.0,935.0,935.0,825.0,770.0,935.0
2,ID_0IWQNPI,"['02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020']","[2850.0, 1500.0, 1350.0, 610.0, 200.0, 250.0]",700.0,1350.0,1550.0,1400.0,1450.0,1200.0
3,ID_IY8SYB9,"['09-2017', '10-2017', '11-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018']","[2200.0, 1420.0, 1180.0, 900.0, 1400.0, 780.0, 1300.0, 1100.0, 600.0, 380.0]",580.0,480.0,800.0,1260.0,1650.0,530.0
4,ID_9XHL7VZ,"['09-2017', '10-2017', '11-2017', '12-2017', '01-2018', '02-2018', '03-2018', '04-2018', '05-2018', '06-2018', '07-2018', '08-2018', '10-2018', '11-2018', '12-2018', '02-2019', '03-2019', '07-2019', '08-2019', '09-2019', '10-2019', '11-2019']","[2640.0, 910.0, 480.0, 280.0, 200.0, 180.0, 330.0, 200.0, 370.0, 289.0, 80.0, 660.0, 40.0, 160.0, 80.0, 80.0, 40.0, 80.0, 40.0, 40.0, 80.0, 520.0]",40.0,440.0,460.0,360.0,80.0,330.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28007 entries, 0 to 28006
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                28007 non-null  object 
 1   TransactionDates  28007 non-null  object 
 2   PaymentsHistory   28007 non-null  object 
 3   m1                28007 non-null  float64
 4   m2                28007 non-null  float64
 5   m3                28007 non-null  float64
 6   m4                28007 non-null  float64
 7   m5                28007 non-null  float64
 8   m6                28007 non-null  float64
dtypes: float64(6), object(3)
memory usage: 1.9+ MB


In [ ]:
#total no.of null values in each column
mdata.isnull().sum(axis = 0)

ID                         0
RegistrationDate           0
Deposit                    0
UpsellDate             36370
AccessoryRate              0
PaymentMethod              0
rateTypeEntity             0
RatePerUnit                0
DaysOnDeposit              0
MainApplicantGender        0
Age                     6939
Region                  1934
Town                       0
Occupation                 0
SupplierName               0
Term                       0
TotalContractValue         0
ExpectedTermDate           0
FirstPaymentDate           0
LastPaymentDate            0
dtype: int64

# Pre-processing dataset

In [ ]:
#merging with test dataset as it also contains region
pd.merge(left=mdata[mdata.isna().Region], right=test, on="ID")
mdata.head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_K00S4N4,2015-12-10 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30
1,ID_6L67PAA,2015-12-09 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740.0,2016-12-07 00:00:00,2015-12-09 13:14:03,2020-05-24 15:32:18
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,43.0,NaN,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,NaN,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52


In [ ]:
#filling null values
mdata["UpsellDate"]=mdata["UpsellDate"].fillna(0)
mdata["Region"]=mdata["Region"].fillna('other')

In [ ]:
#merging test dataset 
pd.merge(left=mdata[mdata.isna().Age], right=test, on="ID")
mdata.head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate
0,ID_K00S4N4,2015-12-10 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30
1,ID_6L67PAA,2015-12-09 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740.0,2016-12-07 00:00:00,2015-12-09 13:14:03,2020-05-24 15:32:18
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,43.0,other,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52


In [ ]:
#using mean to replace null values in age column
x = mdata["Age"].mean()
mdata["Age"].fillna(x, inplace = True)

In [ ]:
#merging all datasets and replacing null values with 0
mer = pd.merge(left=mdata, right=pd.concat(objs=[train, test]).fillna(value=0), on="ID")
mer.head(5)

,ID,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6
0,ID_K00S4N4,2015-12-10 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,364,14740.0,2016-12-08 00:00:00,2015-12-10 09:52:35,2016-10-23 04:52:30,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016']","[3050.0, 1050.0, 910.0, 1050.0, 1050.0]",1225.0,1050.0,1190.0,525.0,1750.0,1890.0
1,ID_6L67PAA,2015-12-09 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,364,14740.0,2016-12-07 00:00:00,2015-12-09 13:14:03,2020-05-24 15:32:18,"['12-2015', '01-2016', '02-2016', '03-2016', '05-2016', '07-2016']","[4000.0, 1050.0, 1050.0, 1050.0, 1050.0, 400.0]",0.0,0.0,0.0,0.0,0.0,0.0
2,ID_102CV85,2015-12-18 00:00:00,2000,2018-03-29 10:14:58,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,392,29480.0,2017-01-13 00:00:00,2015-12-18 06:22:34,2017-02-01 15:23:44,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016']","[4245.0, 980.0, 735.0, 735.0, 1470.0, 735.0, 980.0, 915.0, 735.0]",980.0,980.0,1225.0,980.0,935.0,355.0
3,ID_HXBJFHB,2015-11-25 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,43.0,other,UNKNOWN,Teacher,d_light,364,14740.0,2016-11-23 00:00:00,2015-11-25 13:25:57,2017-05-22 16:46:54,"['11-2015', '12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '10-2016']","[2245.0, 980.0, 980.0, 1225.0, 980.0, 980.0, 980.0, 1225.0, 735.0, 490.0, 250.0]",250.0,1000.0,250.0,500.0,560.0,1150.0
4,ID_3K9VZ5J,2015-12-02 00:00:00,2000,0,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,364,14740.0,2016-11-30 00:00:00,2015-12-05 10:34:32,2017-05-12 16:50:52,"['12-2015', '01-2016', '02-2016', '03-2016', '04-2016', '05-2016', '06-2016', '07-2016', '08-2016', '09-2016', '10-2016', '11-2016']","[2750.0, 1000.0, 750.0, 1000.0, 955.0, 880.0, 280.0, 665.0, 770.0, 420.0, 525.0, 735.0]",630.0,805.0,700.0,855.0,245.0,775.0


In [ ]:
#droping fisrtpaymentdate column
mer= mer.drop(["FirstPaymentDate"],axis=1)

In [ ]:
#function to convert the period between dates to months
def period(a,b):
    start=b.split('-')
    end=a.split('-')
    return int(end[1])-int(start[1])+12*(int(end[0])-int(start[0]))

In [ ]:
#converting the period between various dates and registrationdate to months
#payment of money divided by initial amount
mer["time"] = np.nan
mer["PaidAmount"]=np.nan
mer["PeriodDiff"]=np.nan
#for index, row in mer.iterrows():
for index in tqdm(range(0,mer.shape[0]),desc='Row'):
    year=int(mer.RegistrationDate.iloc[index].split('-')[0])
    month=int(mer.RegistrationDate.iloc[index].split('-')[1])
    time=np.ceil(mer.loc[index,'Term']/30)
    mer.loc[index,'time']=time
    if mer.UpsellDate.iloc[index]!=0:
        mer.loc[index,'UpsellDate']=period(mer.UpsellDate.iloc[index],mer.RegistrationDate.iloc[index])/time
    mer.loc[index,'LastPaymentDate']=period(mer.LastPaymentDate.iloc[index],mer.RegistrationDate.iloc[index])/time
    mer.loc[index,'ExpectedTermDate']=period(mer.ExpectedTermDate.iloc[index],mer.RegistrationDate.iloc[index])/time   
    dates=[]
    for i in mer.loc[index,'TransactionDates'].split("'"):
        if '-' in i:
            date=i.split('-')
            dates.append((int(date[0])-month+12*(int(date[1])-year))/time)
    mer.at[index,'TransactionDates']=dates

    mer.loc[index,'m1']=mer.loc[index,'m1']/mer.loc[index,'TotalContractValue']
    mer.loc[index,'m2']=mer.loc[index,'m2']/mer.loc[index,'TotalContractValue']
    mer.loc[index,'m3']=mer.loc[index,'m3']/mer.loc[index,'TotalContractValue']
    mer.loc[index,'m4']=mer.loc[index,'m4']/mer.loc[index,'TotalContractValue']
    mer.loc[index,'m5']=mer.loc[index,'m5']/mer.loc[index,'TotalContractValue']
    mer.loc[index,'m6']=mer.loc[index,'m6']/mer.loc[index,'TotalContractValue']
    amount=[]
    for i in mer.loc[index,'PaymentsHistory'].split("[")[1].split("]")[0].split(","):
        if '.' in i:
            amount.append(float(i)/mer.loc[index,'TotalContractValue'])
    mer.at[index,'PaymentsHistory']=amount
    mer.loc[index,'PaidAmount']=sum(amount)
    mer.loc[index,'PeriodDiff']=mer.loc[index,'LastPaymentDate']-dates[-1]
mer= mer.drop(["RegistrationDate","Term"],axis=1)  

Row: 100%|██████████| 37343/37343 [06:34<00:00, 94.58it/s]


In [ ]:
mer.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0.0,FINANCED,DAILY,35,7,Male,41.0,Mount Kenya Region,Embu,Other,d_light,14740.0,0.923077,0.769231,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13.0,0.482361,0.461538
1,ID_6L67PAA,2000,0,0.0,FINANCED,DAILY,35,7,Male,33.0,Coast Region,Kilifi,Other,d_light,14740.0,0.923077,4.07692,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.583446,3.538462
2,ID_102CV85,2000,1.92857,35.0,FINANCED,DAILY,35,7,Female,48.0,Nairobi Region,Makueni,Business,d_light,29480.0,0.928571,1,"[0.0, 0.07142857142857142, 0.14285714285714285, 0.21428571428571427, 0.2857142857142857, 0.35714285714285715, 0.42857142857142855, 0.5, 0.5714285714285714]","[0.14399592944369063, 0.03324287652645862, 0.024932157394843963, 0.024932157394843963, 0.049864314789687926, 0.024932157394843963, 0.03324287652645862, 0.031037991858887382, 0.024932157394843963]",0.033243,0.033243,0.041554,0.033243,0.031716,0.012042,14.0,0.391113,0.428571
3,ID_HXBJFHB,2000,0,0.0,FINANCED,DAILY,35,7,Female,43.0,other,UNKNOWN,Teacher,d_light,14740.0,0.923077,1.38462,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.8461538461538461]","[0.1523066485753053, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.06648575305291723, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.049864314789687926, 0.03324287652645862, 0.016960651289009497]",0.016961,0.067843,0.016961,0.033921,0.037992,0.078019,13.0,0.751018,0.538462
4,ID_3K9VZ5J,2000,0,0.0,FINANCED,DAILY,35,7,Female,56.0,Mount Kenya Region,Kirinyaga,Other,d_light,14740.0,0.846154,1.30769,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.7692307692307693, 0.8461538461538461]","[0.1865671641791045, 0.06784260515603799, 0.05088195386702849, 0.06784260515603799, 0.06478968792401628, 0.05970149253731343, 0.018995929443690638, 0.04511533242876527, 0.05223880597014925, 0.028493894165535955, 0.03561736770691994, 0.049864314789687926]",0.042741,0.054613,0.047490,0.058005,0.016621,0.052578,13.0,0.727951,0.461538


In [ ]:
mer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37343 entries, 0 to 37342
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   37343 non-null  object 
 1   Deposit              37343 non-null  int64  
 2   UpsellDate           37343 non-null  object 
 3   AccessoryRate        37343 non-null  float64
 4   PaymentMethod        37343 non-null  object 
 5   rateTypeEntity       37343 non-null  object 
 6   RatePerUnit          37343 non-null  int64  
 7   DaysOnDeposit        37343 non-null  int64  
 8   MainApplicantGender  37343 non-null  object 
 9   Age                  37343 non-null  float64
 10  Region               37343 non-null  object 
 11  Town                 37343 non-null  object 
 12  Occupation           37343 non-null  object 
 13  SupplierName         37343 non-null  object 
 14  TotalContractValue   37343 non-null  float64
 15  ExpectedTermDate     37343 non-null 

**Label Encoding**

In [ ]:
#label encoding
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder()
obj_list=['PaymentMethod','rateTypeEntity','MainApplicantGender','Region','Town','Occupation','SupplierName']
for x in obj_list:
    mer[x] = label_encoder.fit_transform(mer[x])
mer[obj_list].nunique()  

PaymentMethod           1
rateTypeEntity          3
MainApplicantGender     2
Region                  8
Town                   48
Occupation              7
SupplierName            1
dtype: int64

In [ ]:
# As these two variable have only one class.
mer= mer.drop(['PaymentMethod','SupplierName'],axis=1)

In [ ]:
mer.head(5)

,ID,Deposit,UpsellDate,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,TotalContractValue,ExpectedTermDate,LastPaymentDate,TransactionDates,PaymentsHistory,m1,m2,m3,m4,m5,m6,time,PaidAmount,PeriodDiff
0,ID_K00S4N4,2000,0,0.0,0,35,7,1,41.0,1,5,5,14740.0,0.923077,0.769231,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077]","[0.20691994572591588, 0.07123473541383989, 0.06173677069199457, 0.07123473541383989, 0.07123473541383989]",0.083107,0.071235,0.080733,0.035617,0.118725,0.128223,13.0,0.482361,0.461538
1,ID_6L67PAA,2000,0,0.0,0,35,7,1,33.0,0,13,5,14740.0,0.923077,4.07692,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.38461538461538464, 0.5384615384615384]","[0.27137042062415195, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.07123473541383989, 0.027137042062415198]",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,0.583446,3.538462
2,ID_102CV85,2000,1.92857,35.0,0,35,7,0,48.0,2,22,0,29480.0,0.928571,1,"[0.0, 0.07142857142857142, 0.14285714285714285, 0.21428571428571427, 0.2857142857142857, 0.35714285714285715, 0.42857142857142855, 0.5, 0.5714285714285714]","[0.14399592944369063, 0.03324287652645862, 0.024932157394843963, 0.024932157394843963, 0.049864314789687926, 0.024932157394843963, 0.03324287652645862, 0.031037991858887382, 0.024932157394843963]",0.033243,0.033243,0.041554,0.033243,0.031716,0.012042,14.0,0.391113,0.428571
3,ID_HXBJFHB,2000,0,0.0,0,35,7,0,43.0,7,43,6,14740.0,0.923077,1.38462,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.8461538461538461]","[0.1523066485753053, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.06648575305291723, 0.06648575305291723, 0.06648575305291723, 0.08310719131614654, 0.049864314789687926, 0.03324287652645862, 0.016960651289009497]",0.016961,0.067843,0.016961,0.033921,0.037992,0.078019,13.0,0.751018,0.538462
4,ID_3K9VZ5J,2000,0,0.0,0,35,7,0,56.0,1,14,5,14740.0,0.846154,1.30769,"[0.0, 0.07692307692307693, 0.15384615384615385, 0.23076923076923078, 0.3076923076923077, 0.38461538461538464, 0.46153846153846156, 0.5384615384615384, 0.6153846153846154, 0.6923076923076923, 0.7692307692307693, 0.8461538461538461]","[0.1865671641791045, 0.06784260515603799, 0.05088195386702849, 0.06784260515603799, 0.06478968792401628, 0.05970149253731343, 0.018995929443690638, 0.04511533242876527, 0.05223880597014925, 0.028493894165535955, 0.03561736770691994, 0.049864314789687926]",0.042741,0.054613,0.047490,0.058005,0.016621,0.052578,13.0,0.727951,0.461538


In [ ]:
#saving the final merged dataset
mer.to_csv("final.csv",index=False)

# splitting dataset

In [ ]:
df_final=pd.DataFrame(columns=["time","money","id"])
for k in tqdm(range(0,mer.shape[0])):
    df_temp=pd.DataFrame({"time":mer.loc[k,"TransactionDates"],"money":mer.loc[k,"PaymentsHistory"]})
    df_temp['id']=mer.loc[k,"ID"]
    df_final=pd.concat([df_final, df_temp], ignore_index=True)
print(df_final.shape)

100%|██████████| 37343/37343 [03:42<00:00, 167.80it/s]

(607851, 3)


In [ ]:
df_final.head()

,time,money,id
0,0.000000,0.206920,ID_K00S4N4
1,0.076923,0.071235,ID_K00S4N4
2,0.153846,0.061737,ID_K00S4N4
3,0.230769,0.071235,ID_K00S4N4
4,0.307692,0.071235,ID_K00S4N4


In [ ]:
#extracting features
data=pd.DataFrame()
from tsfresh import extract_features
k=10000
for i in range(0,len(df_final.index),10000):
  if len(df_final.index) - i < 10000:
    k = len(df_final.index) - i 
  extracted_features = extract_features(df_final[:][i:i+k],column_id='id',column_sort="time").dropna(axis=1)
  data=pd.concat([data,extracted_features], ignore_index=True)


Feature Extraction: 100%|██████████| 5/5 [00:54<00:00, 10.98s/it]


In [ ]:
data = data.fillna(0)

In [ ]:
#flitering extracted features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
y = mer["m1"]
impute(data)
features_filtered = select_features(data, y)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:9011: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cond, other, inplace, axis, level, errors=errors, try_cast=try_cast


In [ ]:
features_filtered.head(5)

,money__large_standard_deviation__r_0.25,money__maximum,"money__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","money__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0","money__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)",money__standard_deviation,money__variance,money__time_reversal_asymmetry_statistic__lag_1,money__large_standard_deviation__r_0.30000000000000004,money__root_mean_square,money__quantile__q_0.9,"money__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",money__time_reversal_asymmetry_statistic__lag_2,"money__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.0","money__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.0",money__abs_energy,money__range_count__max_1__min_-1,money__range_count__max_1000000000000.0__min_0,money__length,money__quantile__q_0.8,money__cid_ce__normalize_True,money__lempel_ziv_complexity__bins_3,money__index_mass_quantile__q_0.1,"money__fft_aggregated__aggtype_""variance""",money__count_below_mean,money__lempel_ziv_complexity__bins_2,money__mean,money__has_duplicate,money__lempel_ziv_complexity__bins_5,money__last_location_of_maximum,money__c3__lag_1,money__number_peaks__n_1,money__quantile__q_0.7,money__number_cwt_peaks__n_1,money__lempel_ziv_complexity__bins_10,"money__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",money__energy_ratio_by_chunks__num_segments_10__segment_focus_9,money__ratio_beyond_r_sigma__r_3,money__quantile__q_0.6,"money__fft_aggregated__aggtype_""centroid""",money__large_standard_deviation__r_0.2,money__number_peaks__n_3,money__c3__lag_2,money__symmetry_looking__r_0.1,money__ratio_beyond_r_sigma__r_1.5,money__count_above_mean,"money__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2",money__symmetry_looking__r_0.05,money__median,"money__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2",money__cid_ce__normalize_False,money__number_cwt_peaks__n_5,money__quantile__q_0.4,"money__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2","money__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)",money__lempel_ziv_complexity__bins_100,money__energy_ratio_by_chunks__num_segments_10__segment_focus_8,"money__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",money__approximate_entropy__m_2__r_0.3,money__quantile__q_0.3,money__number_peaks__n_5,"money__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4",money__fourier_entropy__bins_100,"money__agg_autocorrelation__f_agg_""median""__maxlag_40",money__quantile__q_0.2,"money__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.4","money__linear_trend__attr_""rvalue""","money__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0",money__large_standard_deviation__r_0.35000000000000003,money__longest_strike_above_mean,money__longest_strike_below_mean,"money__agg_autocorrelation__f_agg_""mean""__maxlag_40","money__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)",money__sum_of_reoccurring_data_points,money__minimum,money__quantile__q_0.1,money__mean_change,Unnamed: 0,"money__fft_aggregated__aggtype_""skew""",money__ratio_value_number_to_time_series_length,"money__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","money__fft_aggregated__aggtype_""kurtosis""","money__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0",money__energy_ratio_by_chunks__num_segments_10__segment_focus_7,"money__linear_trend__attr_""slope""","money__change_quantiles__f_agg_""var""__isabs_True__qh_0.2__ql_0.0","money__cwt_coefficients__coeff_1__w_5__widths_(2, 5, 10, 20)",money__sum_of_reoccurring_values,money__percentage_of_reoccurring_datapoints_to_all_datapoints,money__approximate_entropy__m_2__r_0.5,"money__linear_trend__attr_""stderr""","money__linear_trend__attr_""intercept""",money__permutation_entropy__dimension_5__tau_1,"money__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2",money__mean_abs_change,money__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"money__cha

Training and Testing Model

In [ ]:
#k-cross validation
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
features_filtered["m1"] = list(mer["m1"])
result = next(kf.split(features_filtered), None)
train = features_filtered.iloc[result[0]]
test =  features_filtered.iloc[result[1]]

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [ ]:
#LightGBM regression
n_HP_points_to_test = 100
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lightgbm import LGBMRegressor
model = LGBMRegressor()
gs = RandomizedSearchCV(
    model,param_distributions=param_test,
    n_iter=n_HP_points_to_test)

In [ ]:
y = train["m1"]
actual = test["m1"]
train=train.drop("m1",axis=1)
test=test.drop("m1",axis=1)

In [ ]:
gs.fit(train, y)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=None,
                                           reg_alpha=0.0, reg_lambda=0.0,
                                           si...
                                                             100.0, 1000.0,
                                                            

In [ ]:
#function to find error
from sklearn.metrics import mean_squared_error 
RMSE=[]
def error(actual,prediction):
 rmse = mean_squared_error(actual, prediction, squared=False)
 RMSE.append(math.sqrt(mean_squared_error(actual, prediction)))
 return(rmse)

In [ ]:
prediction = gs.predict(test)
print("LightGBM Regression for m1:\n RMSE:", error(actual,prediction))

LightGBM Regression for m1:
 RMSE: 0.029909152103850783


In [ ]:
#m2
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import math
y = mer["m2"]
impute(data)
features_filtered = select_features(data, y)

from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
features_filtered["m2"] = list(mer["m2"])
result = next(kf.split(features_filtered), None)
train = features_filtered.iloc[result[0]]
test =  features_filtered.iloc[result[1]]
n_HP_points_to_test = 100

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lightgbm import LGBMRegressor
model = LGBMRegressor()
gs = RandomizedSearchCV(
    model,param_distributions=param_test,
    n_iter=n_HP_points_to_test)
y = train["m2"]
actual = test["m2"]
train=train.drop("m2",axis=1)
test=test.drop("m2",axis=1)

gs.fit(train, y)
prediction = gs.predict(test)
print("LightGBM Regression for m2:\n RMSE:", error(actual,prediction))

LightGBM Regression for m2:
 RMSE: 0.029031782230100348


In [ ]:
#m3
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import math
y = mer["m3"]
impute(data)
features_filtered = select_features(data, y)

from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
features_filtered["m3"] = list(mer["m3"])
result = next(kf.split(features_filtered), None)
train = features_filtered.iloc[result[0]]
test =  features_filtered.iloc[result[1]]
n_HP_points_to_test = 100

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lightgbm import LGBMRegressor
model = LGBMRegressor()
gs = RandomizedSearchCV(
    model,param_distributions=param_test,
    n_iter=n_HP_points_to_test)
y = train["m3"]
actual = test["m3"]
train=train.drop("m3",axis=1)
test=test.drop("m3",axis=1)

gs.fit(train, y)
prediction = gs.predict(test)
print("LightGBM Regression for m3:\n RMSE:", error(actual,prediction))

LightGBM Regression for m3:
 RMSE: 0.02921300224456549


In [ ]:
#m4
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import math
y = mer["m4"]
impute(data)
features_filtered = select_features(data, y)

from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
features_filtered["m4"] = list(mer["m4"])
result = next(kf.split(features_filtered), None)
train = features_filtered.iloc[result[0]]
test =  features_filtered.iloc[result[1]]
n_HP_points_to_test = 100

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lightgbm import LGBMRegressor
model = LGBMRegressor()
gs = RandomizedSearchCV(
    model,param_distributions=param_test,
    n_iter=n_HP_points_to_test)
y = train["m4"]
actual = test["m4"]
train=train.drop("m4",axis=1)
test=test.drop("m4",axis=1)

gs.fit(train, y)
prediction = gs.predict(test)
print("LightGBM Regression for m4:\n RMSE:", error(actual,prediction))

LightGBM Regression for m4:
 RMSE: 0.03000722650228585


In [ ]:
#m5
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import math
y = mer["m5"]
impute(data)
features_filtered = select_features(data, y)

from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
features_filtered["m5"] = list(mer["m5"])
result = next(kf.split(features_filtered), None)
train = features_filtered.iloc[result[0]]
test =  features_filtered.iloc[result[1]]
n_HP_points_to_test = 100

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lightgbm import LGBMRegressor
model = LGBMRegressor()
gs = RandomizedSearchCV(
    model,param_distributions=param_test,
    n_iter=n_HP_points_to_test)
y = train["m5"]
actual = test["m5"]
train=train.drop("m5",axis=1)
test=test.drop("m5",axis=1)

gs.fit(train, y)
prediction = gs.predict(test)
print("LightGBM Regression for m5:\n RMSE:", error(actual,prediction))

LightGBM Regression for m5:
 RMSE: 0.03097283171550951


In [ ]:
#m6
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import math
y = mer["m6"]
impute(data)
features_filtered = select_features(data, y)

from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
features_filtered["m6"] = list(mer["m6"])
result = next(kf.split(features_filtered), None)
train = features_filtered.iloc[result[0]]
test =  features_filtered.iloc[result[1]]
n_HP_points_to_test = 100

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from lightgbm import LGBMRegressor
model = LGBMRegressor()
gs = RandomizedSearchCV(
    model,param_distributions=param_test,
    n_iter=n_HP_points_to_test)
y = train["m6"]
actual = test["m6"]
train=train.drop("m6",axis=1)
test=test.drop("m6",axis=1)

gs.fit(train, y)
prediction = gs.predict(test)
print("LightGBM Regression for m6:\n RMSE:", error(actual,prediction))

LightGBM Regression for m6:
 RMSE: 0.04084088851288605
